## 코루틴 (Coroutine)의 정의
* 특정 위치에서 실행을 일시 중단하고 다시 시작할 수 있는 여러 진입점을 허용하여 **비선점** 멀티 태스킹을 위해 서브루틴을 일반화하는 컴퓨터 프로그램 구성 요소
- - - 
* 선점형 (preemption) vs 비선점형 (non-preemption)
    * 여러 프로세스 동시에 돌아가는 것은 실질적으로 동시라기보다 시분할으로 봐야 함
    * 선점형
        * 하나의 프로세스가 다른 프로세스 대신에 프로세서 (CPU) 차지 가능   
        ![preemtive](../images/chap16-2_preemptive.png)
    * 비선점형
        * 하나의 프로세스가 끝나지 않으면 다른 프로세스 CPU 사용 불가   
        ![non-preemtive](../images/chap16-2_non-preemptive.png)
    * [출처](https://maxpulse.tistory.com/208)
    * [그림 출처](https://www.geeksforgeeks.org/preemptive-and-non-preemptive-scheduling/)
* 서브루틴
    * 진입점과 중단점이 1개인 코루틴
    * 함수
- - -
* [출처](https://www.slideshare.net/QooJuice/coroutine-119750550)

## yield from의 의미

본격적으로 시작하기 앞서...    

from chapter16-1_jinny.ipynb   

### Coroutine syntax changes
1. 이전에는 generator 내에서 값을 return하면 SyntaxError를 발생시켰는데, 현재는 가능해졌다.  
2. `yield from`을 사용해서 복잡한 generator를 작게 쪼개서 사용할 수 있다. (리팩토링!)    

... 중략 ....    

###  `yield from` 사용

... 중략 ....     

2가지 쓰임  
1. for문을 사용하지 않고 값을 여러 번 바깥으로 전달 가능(generator에서의 쓰임)  
2. `yield from`에 coroutine을 지정하면 해당 coroutine에서 return으로 반환한 값을 가져온다. (Syntax for Delegating to a Subgenerator)


* `yield from`의 등장 배경
    * 코루틴의 하나인 파이썬 제너레이터는 직전 호출자 (immediate caller)에 yield할 수 밖에 없는 제한점이 있음
    * 위는 ```yield``` 구문을 포함할 경우 다른 코드와 같이 분리되어 별도의 함수에 넣을 수 없음을 의미
    * ```yield``` 구문을 포함한 구문을 함수로 분리할 경우 함수 자체가 제너레이터가 되어 값은 다시 yield해야 함
    * [출처](https://www.python.org/dev/peps/pep-0380/)

In [1]:
def gen():
    for i in range(3):
        yield i
        
print(list(gen()))

def gen2():
    yield from range(3)
    
print(list(gen()))

[0, 1, 2]
[0, 1, 2]


* 특징
    * 하위 제너레이터가 생성한 값은 모두 대표 제너레이터의 호출자에 바로 전달
    * `send()`를 통해 대표 제너레이터에 전달한 값은 모두 하위 제너레이터에 직접 전달
        * 값이 `None`일 경우 하위 제너레이터의 `__next__()` 호출
        * 이외의 경우 `send()` 호출
        * 호출된 메서드에서 `StopIteration` 예외 발생 시 대표 제너레이터 실행 재개
    * 제너레이터나 하위 제너레이터에 `return expr`문 실행 시 제너레이터 빠져나와 `StopIteration(expr)` 예외 발생
    * 하위 제너레이터 실행 마친 후 발생한 `StopIteration` 예외 첫 인수가 `yield from`의 값이 됨
    * 대표 제너레이터에 던져진 `GeneratorExit` 예외는 하위 제너레이터의 `throw()` 메서드에 전달
        * `throw()` 메소드 호출해 `StopIteration` 예외 발생 시 대표 제너레이터 실행 재개
        * 그 외 예외 대표 제너레이터에 전달
    * `GeneratorExit` 예외가 대표 제너레이터에 던져지거나 대표 제너레이터의 `close()` 메서드 호출되면 하위 제너레이터의 `close()` 메서드 호출
        * 그 결과로 예외 발생하면 발생 예외가 대표 제너레이터에 전파
        * 그렇지 않을 경우 `GeneratorExit` 예외 발생

In [ ]:
# RESULT = yield from EXPR
_i = iter(EXPR) # _i 하위 제너레이터
try:
    # _y 하위 제너레이터 생성 값
    # 하위 제너레이터 가동
    _y = next(_i)
# StopIteration 예외 발생 시 예외 객체 값이 최종 결과값 _r에 할당
except StopItertaion as _e:
    _r = _e.value
else:
    # 대표 제너레이터 실행 중단, 하위 제너레이터와 호출자 간 통로 역할만 수행
    while 1:
        try:
            # 하위 제너레이터 값 생성
            _s = yield _y
        except GeneratorExit as _e:
            # _i = iter(EXPR)에 의해 모든 반복형이 하위 제너레이터가 되어 close()메서드 없을 수도 있음
            try:
                _m = _i.close
            except AttributeError:
                pass
            else:
                _m()
            raise _e
        # Python 모든 내장 에러는 BaseException의 서브 클래스
        # 시스템 에러 제외 모든 내장 에러 Exception 클래스의 서브 클래스
        # https://nachwon.github.io/exception/
        except BaseException as _e:
            '''
            print "0", sys.exc_info()[0]: <type 'exceptions.NameError'>
            print "1", sys.exc_info()[1]: 
            print "2", sys.exc_info()[2]: <traceback object at 0xbd5fc8>
            '''
            _x = sys.exec_info()
            # 호출자가 throw()로 던진 예외 처리
            # 하위 제너레이터가 throw() 구현하지 않을 수 있는데, 이 경우 대표 제너레이터에서 예외 발생
            try:
                _m = _i.throw
            except AttributeError as _e:
                raise _e
            # 하위 제너레이터가 throw() 메서드 가질 경우 호출자로부터 받은 예외 이용해 호출
            # 하위 제너레이터는 예외 처리하거나 (루프 계속 실행)
            # StopIteration 예외 발생 (결과 추출 후 루프 종료)
            # 처리할 수 없는 예외 발생시킬 수 있음 (대표 제너레이터로 전파)
            else:
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        # 예외 발생 안 한 경우
        else:
            try:
                if _s is None:
                    _y = next(_i)
                else:
                    _y = _i.send(_i)
                except StopIteration as _e:
                    _r = _e.value
                    break
        # _r이 전체 yield from 표현식 값이 되어 RESULT에 저장
        RESULT = _r

## 사용 사례: 이산 이벤트 시뮬레이션을 위한 코루틴
* 코루틴은 시뮬레이션, 게임, 비동기 입출력, 그외 이벤트 주도 프로그래밍이나 협업적 멀티테스킹 등의 알고리즘 표현
- - - 
* 협업적 멀티테스킹
    * 일종의 시분할 방식
    * 운영체제의 개입 없이 task가 독점적으로 CPU 사용
    * 미사용 시 자발적 CPU 자원 반환
    * Critical section 보호를 위한 Lock이나 Semaphore 불필요
    * [출처](http://playnode.io/2017/slides/nodejs-cooperative-multi-tasking.pdf)
* Lock vs Semaphore
    * Lock 
        * 0 또는 1
        * 사용했을 경우 1, 아닐 경우 0
    * Semaphore
        * 기찻길에서 깃발 표식으로 지나가도 되는지 여부를 표시하는 것으로부터 유래
        * shared data의 개수 의미 (0, 1, 2, ... 값 가능)
        * (0 또는 1의 값만 갖는 binary semaphore의 경우) 초기화 1이고, 누가 사용할 경우 0
        * (0 또는 1의 값 외 값을 갖는 counting semaphore의 경우) 공유자원 개수를 나태냄
            * 사용 가능한 자원이 없을 경우 0, 사용 후 반환하면 semaphore 1 증가
    * [출처](https://jhnyang.tistory.com/101)
- - -
* 이산 이벤트 시뮬레이션 (Discrete Event Simulation, DES)
    * 시스템을 일련의 이벤트로 모델링
    * 시뮬레이션 시간은 모델링된 다음 이벤트의 시뮬레이션된 시각으로 바로 진행

In [ ]:
# https://github.com/fluentpython/example-code/tree/master/16-coroutine
import collections


Event = collections.nametuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')

In [ ]:
class Simulator:

    def __init__(self, procs_map):
        # Event 객체 보관
        # PriorityQueue 항목 넣고 정렬된 순서대로 꺼내옴
        self.events = queue.PriorityQueue()
        # 프로세스 번호를 시뮬레이션의 활성화된 프로세스로 매핑
        # 택시 하나를 나타내는 제너레이터 객체를 하나의 프로세스로 표현
        self.procs = dict(procs_map)

    def run(self, end_time):  # <1>
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):  # <2>
            first_event = next(proc)  # <3>
            self.events.put(first_event)  # <4>

        # main loop of the simulation
        sim_time = 0  # <5>
        while sim_time < end_time:  # <6>
            if self.events.empty():  # <7>
                print('*** end of events ***')
                break

            current_event = self.events.get()  # <8>
            sim_time, proc_id, previous_action = current_event  # <9>
            print('taxi:', proc_id, proc_id * '   ', current_event)  # <10>
            active_proc = self.procs[proc_id]  # <11>
            next_time = sim_time + compute_duration(previous_action)  # <12>
            try:
                next_event = active_proc.send(next_time)  # <13>
            except StopIteration:
                del self.procs[proc_id]  # <14>
            else:
                self.events.put(next_event)  # <15>
        else:  # <16>
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
# END TAXI_SIMULATOR

## 요약
* 세 가지 스타일의 제너레이터 코드가 존재
    * 풀 (반복자)
    * 푸시 (이동 평균)
    * 작업 (시뮬레이션의 택시 프로세스)
* 데커레이터 이용시 코루틴 편리하게 이용 가능
    * `yield from` 하위 제너레이터 자동 기동
* `yield from`을 통해 값 반환 가능 (`return` 시 `StopIteration` 발생)
    * 대표 제너레이터 본체 안에 yield from 사용
    * 하위 제너레이터 `yield from`으로 활성화
    * 클라이언트 코드 `yield from`으로 설정된 통로 통해 하위 제너레이터로 값 전송